In [1]:
import numpy as np
import face_detection
import os, python_speech_features
import scipy.io.wavfile as wav
import pandas as pd
import torch.nn.functional as F
from talkNet import talkNet
from dataset import MyDataset
from torch.utils.data import DataLoader
import tqdm
import torch
import cv2
import tools
import subprocess
from collections import defaultdict
import whisper
detector = face_detection.build_detector(
"DSFDDetector", confidence_threshold=.3, nms_iou_threshold=.5) #DSFDDetector

c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [89]:
#inputVideo = "C:/Users/jmmol/Desktop/LIP-RTVE/MP4s/speaker092/speaker092_0002.mp4"
inputVideo = r"inputs/dos_personas_hablando_por_turnos.mp4"
videoDuration = tools.checkVideoDuration(inputVideo)
videoFrames = int(videoDuration*25)
fps = 25
#tools.splitVideo(inputVideo,2,videoDuration)

In [3]:
#Carga vídeo
res, facePos, faceFrames = tools.saveMultiFace(inputVideo,detector,50)
# AUDIO PROCESSING
audioPath = tools.convert_video_to_audio_ffmpeg(inputVideo)
_,sig = wav.read(audioPath)#r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker042_0063.wav") #
# _,sig = wav.read(r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker000_0000.wav") 
# audioPath = r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker000_0000.wav"
audio = python_speech_features.mfcc(sig, 16000, numcep = 13, winlen = 0.025, winstep = 0.010) #ASUME VIDEO A 25 Y AUDIO A 100, MODIFICAR
print("Nº CARAS:",len(res))
for cara in res.keys():
    print(len(res[cara]))


Nº CARAS: 3
244
244
3


In [4]:
print(faceFrames[2])
# cv2.imshow("My Video", res[3][50])
# cv2.waitKey(0)

[24, 45, 46]


In [5]:
model = talkNet()
#model.load_state_dict(torch.load("./exps/exp2/model/model_0002.model"))
model.load_state_dict(torch.load("./exps/exp1/model/model21_0006.model"))
windowSize = 21

06-17 10:11:35 Model para number = 15.01


In [6]:
videoDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/npz"
audioDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/mfccs"
datasetTrain = MyDataset(51,videoDir,audioDir,"testSamples.csv")
item = datasetTrain.__getitem__(4309)
valLoader = DataLoader(dataset=datasetTrain,shuffle=False,batch_size=32,num_workers=14) #Cambiar num_workers

25 100


In [7]:
# correctFrames, totalFrames = 0, 0
# correctSamples, totalSamples = 0, 0
# totalPreds = []
# for num, (audioFeature, visualFeature, labels) in enumerate(tqdm.tqdm(valLoader)):
#     with torch.no_grad():    
#         output = model((audioFeature,visualFeature))
#         labels = labels.cuda()
#         batchPreds = torch.reshape(output[1], labels.shape)
#         #Precision a nivel de video
#         videoPreds = torch.mode(batchPreds,dim=1)[0]
#         totalPreds.extend(videoPreds.detach().cpu().numpy().astype(int))
#         labelMode = torch.mode(labels,dim=1)[0]
#         correctSamples += (videoPreds == labelMode).sum().float()
#         totalSamples += len(labels)
#         # Precision a nivel de frame
#         labels = labels.reshape((-1))
#         correctFrames += (output[1] == labels).sum().float()
#         totalFrames += len(labels) 
# #7865
# print("Frame acc:", correctFrames/totalFrames)
# print("Sample acc:", correctSamples/totalSamples)
center = 47
iAudio = tools.padAudio(audio,1,center,51,len(res[0])).unsqueeze(0)
iVideo = tools.padVideo(res[0],center,51).unsqueeze(0)
# print(iAudio[40],item[0][40])
# print("Audios iguales?",(iAudio == item[0]).sum(),iAudio.shape[0]*iAudio.shape[1])
# print("Videos iguales?",(iVideo == item[1]).sum(),iVideo.shape[0]*112*112)
# print(audio[0],item[0][0])

c:\Users\jmmol\Desktop\COSAS V7\TFM\tools.py:182: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  video = torch.FloatTensor(video)


In [8]:
totalScores = defaultdict(list)
meanWSize = 5
meanSideSize = int((meanWSize-1)/2)
# MEDIA FRAMES
# for actualSpeaker in res.keys():
#     print("SPEAKER", actualSpeaker)
#     for i in range(len(res[actualSpeaker])):
#         #center = i
#         center = faceFrames[actualSpeaker][i]
#         print("FRAME Nº ",center)
#         iAudio = tools.padAudio(audio,1,center,windowSize,videoFrames).unsqueeze(0)
#         iVideo = tools.padVideo(res[actualSpeaker],center-faceFrames[actualSpeaker][0],windowSize).unsqueeze(0)
#         #print("audio shape: ", audio.shape)
#         #print("video shape: ", video.shape)
#         # iAudio = item[0].unsqueeze(0)
#         # iVideo = item[1].unsqueeze(0)
#         print(iAudio[0][0][0:5],iVideo[0][0][0][0:5])
#         scores,labels= model((iAudio,iVideo))
#         #print(output)
#         print("Pred ->",torch.mode(labels)[0],"\tProb Pos ->",torch.mean(scores))
#         predArray[actualSpeaker].append(torch.mean(labels).detach().cpu().numpy())


# SECUENCIAL
sideWindowSize = int((windowSize-1)/2)
print("sidewindow:",sideWindowSize)
for actualSpeaker in res.keys():
    print("SPEAKER", actualSpeaker)
    for i in range(0,len(res[actualSpeaker])+windowSize,windowSize):
        #center = i
        center = faceFrames[actualSpeaker][0]+i
        #print("FRAME Nº ",center)
        iAudio = tools.padAudio(audio,1,center,windowSize,videoFrames).unsqueeze(0)
        iVideo = tools.padVideo(res[actualSpeaker],center-faceFrames[actualSpeaker][0],windowSize).unsqueeze(0)
        #print(iAudio[0][0][0:5],iVideo[0][0][0][0:5])
        scores,labels= model((iAudio,iVideo))
        totalScores[actualSpeaker].extend(scores[:,1].detach().cpu().numpy().tolist())
        #predArray[actualSpeaker].extend(labels.detach().cpu().numpy().tolist())

    # MEAN SLIDING WINDOW
    for i in range(len(totalScores[actualSpeaker])):
        ini = max(0,i-meanSideSize) # No negative values
        end = i+meanSideSize+1 # +1 as python does not take into account last value
        #print(ini,end)
        totalScores[actualSpeaker][i] = np.mean(totalScores[actualSpeaker][ini:end])
    print(totalScores[actualSpeaker])

predArray = defaultdict(list)
thr = 0.5
# APPLY THRESHOLD
for actualSpeaker in res.keys():
    # Delete < 0 frame predictions (initial)
    totalScores[actualSpeaker] = totalScores[actualSpeaker][sideWindowSize:len(res[actualSpeaker])+sideWindowSize]
    # Classify as speaking if score higher than threshold
    for sc in totalScores[actualSpeaker]:
        predLabel = 1 if sc > thr else 0
        predArray[actualSpeaker].append(predLabel)

    print(len(predArray[actualSpeaker]))

sidewindow: 10
SPEAKER 0
[0.00575357535853982, 0.004212664731312543, 0.0037662821006961165, 0.002758789421059191, 0.0031045403233729302, 0.0035456935175694525, 0.004341259452402592, 0.004440342498529702, 0.004939847215346247, 0.005239380619086921, 0.005512945291000574, 0.005961367773812228, 0.007106480810671882, 0.01177507221230488, 0.014857717078708815, 0.015459113801185176, 0.013772441407083887, 0.014352512714952886, 0.015490671161359566, 0.041090964840598244, 0.10894690007821586, 0.22351751527692992, 0.27905774782570597, 0.27317377025282086, 0.20902657760156812, 0.17705058921847608, 0.17368036830911535, 0.17926561870965035, 0.1820821679895762, 0.15575969062536776, 0.12423155708103628, 0.1293954389200192, 0.14793126160312706, 0.19632464894347354, 0.1864345880808472, 0.16434953758778842, 0.1321820748831492, 0.13466173977557677, 0.147777790627401, 0.1370731265367189, 0.10134230076593813, 0.0670497513900873, 0.047765249242841055, 0.039339255020425026, 0.036371279002401065, 0.03008969033

In [72]:
def createVideo(videoName,imgFrames,audioPath,width,height):
    video = cv2.VideoWriter("output.mp4", 0, 25, (width,height))

    for image in imgFrames:
        video.write(image)

    cv2.destroyAllWindows()
    video.release()
    print(' '.join(["ffmpeg","-y","-i",os.getcwd()+"/output.mp4","-i",audioPath,"-map","0:v","-map","1:a", "-c:v", "copy", "-shortest", os.getcwd()+"/output2.mp4"]))
    res = subprocess.check_output(["ffmpeg","-y","-i",os.getcwd()+f"/output.mp4","-i",audioPath,"-map","0:v","-map","1:a", "-c:v", "copy", "-shortest", os.getcwd()+f"/outputs/videos/{videoName}"])
    os.remove(os.getcwd()+f"/output.mp4")
    
    print(res)

# Returns a list of [start,end] timestamps in seconds of the parts where a speaker is speaking with minimum length of minLength frames
def getSpeaking(arr,minLength,fps):
    prev_idx = 0
    posFrames = 0
    idx_list = []
    for i,num in enumerate(arr):
        if num == 1:
            posFrames +=1
        else:
            if i-prev_idx >= minLength:
                idx_list.append((prev_idx/fps,i/fps))
            prev_idx = i
            posFrames = 0
    return idx_list

In [99]:
#Asignación de bounding boxes y probabilidades al video de output
os.makedirs("outputs/videos", exist_ok=True)
cap = cv2.VideoCapture(inputVideo)
videoImages = []
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True:
    ret, image = cap.read()
    videoImages.append(image)
    if ret == False:
        break


for speakerN in totalScores.keys():
    for i, fr in enumerate(faceFrames[speakerN]): #Para cada frame en el que aparezca la cara
        try:
            image = videoImages[fr]
            greenValue = int(255*predArray[speakerN][i])
            redValue = 255-greenValue
            color = (0,greenValue,redValue)
            #print(color)
            xmin,ymin,xmax,ymax = facePos[speakerN][i]
            image = cv2.rectangle(image, (xmin,ymin), (xmax,ymax),color , 1)
            cv2.putText(image, "{:.2f}".format(totalScores[speakerN][i]*100), (xmin, ymin-5), cv2.FONT_HERSHEY_SIMPLEX, 0.35, color, 1)
        except:
            pass

        # cv2.imshow("My Video", image)
        # cv2.waitKey(0)


cap.release()
cv2.destroyAllWindows()
path = os.path.normpath(inputVideo)
videoName = str(path.split(os.sep)[-1])
createVideo(videoName,videoImages,audioPath,width,height)
#print(predArray[3])

245
ffmpeg -y -i c:\Users\jmmol\Desktop\COSAS V7\TFM/output.mp4 -i c:\Users\jmmol\Desktop\COSAS V7\TFM/dos_personas_hablando_por_turnos.wav -map 0:v -map 1:a -c:v copy -shortest c:\Users\jmmol\Desktop\COSAS V7\TFM/output2.mp4
b''


In [91]:
# GUARDAR DATOS 
# COGER SOLO FRAGMENTO DE AUDIO EN EL QUE SE DETECTA HABLANTE
os.makedirs("outputs/npz", exist_ok=True)
transcription = model.transcribe(audioPath, language="es", verbose=True,word_timestamps=True)
np.savez_compressed(r"outputs/npz/"+videoName,facePos=facePos,faceFrames=faceFrames,preds=predArray,transcription=transcription)
model = whisper.load_model("small")

c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\whisper\timing.py:42: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower median kernel implementation...
  warnings.warn(
c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\whisper\timing.py:146: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower DTW implementation...
  warnings.warn(


[00:00.320 --> 00:05.500]  y la población civil de guta oriental en siria sigue sufriendo los efectos de la guerra.


In [101]:
# EXTRAER PARTE APROPIADA DE LA TRANSCRIPCIÓN
wordArr = []
alignArr = []
for seg in transcription["segments"]:
    for w in seg["words"]:
        wordArr.append(w["word"])
        alignArr.append((w["start"],w["end"]))
print(wordArr)
print(alignArr)
pdRows = []

for speakerN in totalScores.keys():
    for (ini,end) in getSpeaking(predArray[speakerN],5,25):
        iniW, endW = -1, -1
        for i, (alignIni, alignEnd) in enumerate(alignArr):
            if iniW == -1 and ini <= alignEnd and ini >= alignIni:
                iniW = i
            if endW == -1 and end <= alignEnd and end >= alignIni:
                endW = i
        if iniW > -1 and endW > -1:
            newRow = {'video':inputVideo, 'speaker':speakerN, 'ini': ini, 'end':end, 'npzPath': r"outputs/npz/"+videoName, 'transcription':''.join(wordArr[iniW:endW+1])}
            pdRows.append(newRow)

df = pd.DataFrame(pdRows,columns=['video', 'speaker', 'ini', 'end', 'npzPath', 'transcription'])
df.to_csv(r"outputs/res.csv")

[' y', ' la', ' población', ' civil', ' de', ' guta', ' oriental', ' en', ' siria', ' sigue', ' sufriendo', ' los', ' efectos', ' de', ' la', ' guerra.']
[(0.32, 0.84), (0.84, 0.94), (0.94, 1.2), (1.2, 1.62), (1.62, 1.86), (1.86, 2.1), (2.1, 2.56), (2.56, 2.66), (2.66, 3.14), (3.14, 3.54), (3.54, 4.22), (4.22, 4.48), (4.48, 5.04), (5.04, 5.1), (5.1, 5.24), (5.24, 5.5)]


In [26]:
loaded = np.load(r"C:\Users\jmmol\Desktop\COSAS V7\TFM\outputs\npz\speaker270_0002.mp4.npz",allow_pickle=True)

In [27]:
loaded["transcription"]

array({'text': ' Hace 25 años la ONU declaró el 3 de diciembre como Día Internacional de las Personas con Discapacidad, con el objetivo de concienciarnos a todos sobre su situación y, en su caso,', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 5.28, 'text': ' Hace 25 años la ONU declaró el 3 de diciembre como Día Internacional de las Personas con', 'tokens': [50364, 389, 617, 3552, 11424, 635, 9299, 52, 16694, 812, 806, 805, 368, 14285, 49772, 2617, 413, 2686, 4844, 13608, 368, 2439, 8443, 296, 416, 50628], 'temperature': 0.0, 'avg_logprob': -0.5163383796566823, 'compression_ratio': 1.3311258278145695, 'no_speech_prob': 0.09169353544712067}, {'id': 1, 'seek': 0, 'start': 5.28, 'end': 9.44, 'text': ' Discapacidad, con el objetivo de concienciarnos a todos sobre su situación y, en su caso,', 'tokens': [50628, 4208, 9485, 326, 4580, 11, 416, 806, 29809, 368, 416, 537, 30322, 24979, 257, 6321, 5473, 459, 29343, 288, 11, 465, 459, 9666, 11, 50836], 'temperature': 0.0, 'avg_logprob'